In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## MLP_InBasicWay

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
data = pd.read_csv('/content/drive/MyDrive/Ai_ProjectData/anemia.csv')

In [18]:
data.head()

,Gender,Hemoglobin,MCH,MCHC,MCV,Result
0,1,14.9,22.7,29.1,83.7,0
1,0,15.9,25.4,28.3,72.0,0
2,0,9.0,21.5,29.6,71.2,1
3,0,14.9,16.0,31.4,87.5,0
4,1,14.7,22.0,28.2,99.5,0


In [19]:
data = pd.read_csv('/content/drive/MyDrive/Ai_ProjectData/anemia.csv')

X = data[['Gender', 'Hemoglobin', 'MCHC', 'MCV', 'MCH']].values  # Features
y = data['Result'].values  # Target (0 or 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [20]:

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)


In [21]:
# Number of input features
input_layer_size = X_train.shape[1]  # 5 features (Gender, Hemoglobin, MCHC, MCV, MCH)
hidden_layer_size = 5  # Num Of hidden layer
output_layer_size = 1  # Output

weights_input_hidden = np.random.rand(input_layer_size, hidden_layer_size)
weights_hidden_output = np.random.rand(hidden_layer_size, output_layer_size)
bias_hidden = np.random.rand(1, hidden_layer_size)
bias_output = np.random.rand(1, output_layer_size)


In [30]:
# Training settings
learning_rate = 0.1
epochs = 100000

for epoch in range(epochs):
    hidden_layer_input = np.dot(X_train, weights_input_hidden) + bias_hidden
    hidden_layer_output = sigmoid(hidden_layer_input)

    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    output = sigmoid(output_layer_input)

    error = y_train.reshape(-1, 1) - output

    d_output = error * sigmoid_derivative(output)
    error_hidden_layer = d_output.dot(weights_hidden_output.T)
    d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)

    weights_hidden_output += hidden_layer_output.T.dot(d_output) * learning_rate
    weights_input_hidden += X_train.T.dot(d_hidden_layer) * learning_rate
    bias_output += np.sum(d_output, axis=0, keepdims=True) * learning_rate
    bias_hidden += np.sum(d_hidden_layer, axis=0, keepdims=True) * learning_rate

    if epoch % 10000 == 0:
        print(f'Epoch {epoch}, Error: {np.mean(np.abs(error))}')


Epoch 0, Error: 0.00013108257841099963
Epoch 10000, Error: 0.00012771406806705483
Epoch 20000, Error: 0.00012457788009748337
Epoch 30000, Error: 0.00012164894767593973
Epoch 40000, Error: 0.00011890579315706886
Epoch 50000, Error: 0.00011632989938298763
Epoch 60000, Error: 0.00011390520928485124
Epoch 70000, Error: 0.00011161772426094207
Epoch 80000, Error: 0.00010945517929267703
Epoch 90000, Error: 0.00010740677818875768


In [23]:
hidden_layer_test = sigmoid(np.dot(X_test, weights_input_hidden) + bias_hidden)
test_output = sigmoid(np.dot(hidden_layer_test, weights_hidden_output) + bias_output)

predicted_labels = np.round(test_output)
accuracy = np.mean(predicted_labels == y_test.reshape(-1, 1))
print(f'Accuracy on test set: {accuracy * 100:.2f}%')


Accuracy on test set: 100.00%


## MLPClassifier

In [27]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


data = pd.read_csv('/content/drive/MyDrive/Ai_ProjectData/anemia.csv')


X = data[['Gender', 'Hemoglobin', 'MCHC', 'MCV', 'MCH']].values  # Features
y = data['Result'].values  # Target

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(5, 5), max_iter=1000, activation='relu', solver='adam', random_state=42)

mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")


Accuracy: 0.9976580796252927
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       245
           1       0.99      1.00      1.00       182

    accuracy                           1.00       427
   macro avg       1.00      1.00      1.00       427
weighted avg       1.00      1.00      1.00       427



In [28]:
print("\nIndividual Predictions for Test Data:")
for i in range(len(X_test)):
    print(f"Test sample {i+1} - Features: {X_test[i]} | Predicted: {y_pred[i]} | Actual: {y_test[i]}")


Individual Predictions for Test Data:
Test sample 1 - Features: [ 0.95930718 -0.36108986 -0.3222151  -0.12703693 -0.95908533] | Predicted: 1 | Actual: 1
Test sample 2 - Features: [-1.04241896  0.70282035  0.39186421 -1.53880507  1.33427395] | Predicted: 0 | Actual: 0
Test sample 3 - Features: [ 0.95930718  0.04420926  0.53468007 -0.8536823   1.2082652 ] | Predicted: 0 | Actual: 0
Test sample 4 - Features: [-1.04241896  1.26010665  0.46327214  1.64805391 -0.80787483] | Predicted: 0 | Actual: 0
Test sample 5 - Features: [-1.04241896 -2.13427356  0.96312766  1.47158289 -0.35424332] | Predicted: 1 | Actual: 1
Test sample 6 - Features: [-1.04241896  1.61474339  0.53468007 -1.03015331  1.38467745] | Predicted: 0 | Actual: 0
Test sample 7 - Features: [ 0.95930718 -1.01970095  1.03453559  0.35047289 -0.48025208] | Predicted: 1 | Actual: 1
Test sample 8 - Features: [ 0.95930718  1.51341861 -0.46503096  1.1290215  -1.05989233] | Predicted: 0 | Actual: 0
Test sample 9 - Features: [ 0.95930718 -0